In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import networkx as nx

# 定义Chua电路方程
def chua_circuit(state, t, params):
    x, y, z = state
    ς, ϱ, a, b = params
    l_x = b * x + 0.5 * (a - b) * (abs(x + 1) - abs(x - 1))
    dx = ς * (y - x - l_x)
    dy = x - y + z
    dz = -ϱ * y
    return [dx, dy, dz]

# 参数设置
ς = 10
ϱ = 18
a = -4/3
b = -3/4
params = (ς, ϱ, a, b)

# 时间设置
t = np.linspace(0, 10, 1000)
dt = t[1] - t[0]

# 领导者网络结构
# 全连接领导者网络（5个节点）
M_full = 5
L_full = 4 * np.eye(M_full) - np.ones((M_full, M_full)) + np.eye(M_full)
L_full = -L_full  # 根据论文中的定义调整符号

# 树状结构领导者网络（论文中给出的矩阵）
L_tree = np.array([
    [2, -1, -1, 0, 0],
    [-1, 2, 0, -1, 0],
    [-1, 0, 2, 0, -1],
    [0, -1, 0, 1, 0],
    [0, 0, -1, 0, 1]
])
L_tree = -L_tree  # 根据论文中的定义调整符号

# 模拟领导者网络动态
def simulate_leaders(L, initial_states):
    M = L.shape[0]
    states = np.zeros((len(t), M, 3))  # 存储每个时间步所有领导者的状态
    
    # 数值积分（欧拉法）
    states[0] = initial_states
    for i in range(1, len(t)):
        for j in range(M):
            current_state = states[i-1, j]
            # Chua电路动态
            chua = chua_circuit(current_state, t[i], params)
            # 耦合项
            coupling = 0
            for k in range(M):
                if k != j:
                    coupling += L[j, k] * (states[i-1, k] - current_state)
            # 更新状态
            new_state = current_state + dt * (np.array(chua) + coupling)
            states[i, j] = new_state
    return states

# 初始状态随机生成
np.random.seed(42)
initial_leaders = np.random.randn(5, 3) * 0.1

# 模拟全连接和树状结构领导者
leaders_full = simulate_leaders(L_full, initial_leaders)
leaders_tree = simulate_leaders(L_tree, initial_leaders)

# 计算平均领导者状态
s_bar_full = np.mean(leaders_full, axis=1)
s_bar_tree = np.mean(leaders_tree, axis=1)

# 生成子网（无标度网络，100节点）
N_sub = 100
subnet = nx.barabasi_albert_graph(N_sub, 2)
L_sub = nx.laplacian_matrix(subnet).toarray().astype(float)

# 选择钉扎节点（度数最高和最低的20%）
degrees = np.array([d for _, d in subnet.degree()])
sorted_indices = np.argsort(degrees)
pinning_nodes = np.concatenate([
    sorted_indices[:10],  # 最低度数的10%
    sorted_indices[-10:]  # 最高度数的10%
])

# 子网动态模拟
c = 0.5  # 耦合强度
D = np.zeros(N_sub)
D[pinning_nodes] = 5.0  # 控制增益

def subnet_dynamics(state, t, s_bar, L_sub, D):
    N = L_sub.shape[0]
    states = state.reshape(N, 3)
    derivatives = np.zeros((N, 3))
    
    for i in range(N):
        # Chua电路动态
        chua = chua_circuit(states[i], t, params)
        # 耦合项
        coupling = -c * np.sum(L_sub[i, :, None] * states, axis=0)
        # 钉扎控制项
        if i in pinning_nodes:
            control = -c * D[i] * (states[i] - s_bar)
        else:
            control = np.zeros(3)
        derivatives[i] = chua + coupling + control
    return derivatives.flatten()

# 初始状态
initial_subnet = np.random.randn(N_sub, 3) * 0.1

# 模拟全连接领导者下的子网
subnet_states_full = odeint(
    subnet_dynamics,
    initial_subnet.flatten(),
    t,
    args=(s_bar_full, L_sub, D)
)

# 模拟树状领导者下的子网
subnet_states_tree = odeint(
    subnet_dynamics,
    initial_subnet.flatten(),
    t,
    args=(s_bar_tree, L_sub, D)
)

# 计算误差（欧氏距离）
error_full = np.linalg.norm(
    subnet_states_full.reshape(len(t), N_sub, 3) - s_bar_full[:, None, :],
    axis=2
)

error_tree = np.linalg.norm(
    subnet_states_tree.reshape(len(t), N_sub, 3) - s_bar_tree[:, None, :],
    axis=2
)

# 绘制结果
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
for i in range(N_sub):
    plt.plot(t, error_full[:, i], alpha=0.1, color='blue')
plt.title('Fully Connected Leaders')
plt.xlabel('Time')
plt.ylabel('Error')

plt.subplot(1, 2, 2)
for i in range(N_sub):
    plt.plot(t, error_tree[:, i], alpha=0.1, color='red')
plt.title('Tree-Style Leaders')
plt.xlabel('Time')

plt.tight_layout()
plt.show()

ValueError: could not broadcast input array from shape (1000,3) into shape (3,)